## 454. 4Sum II
- Description:
  <blockquote>
    Given four integer arrays `nums1`, `nums2`, `nums3`, and `nums4` all of length `n`, return the number of tuples `(i, j, k, l)` such that:
   
  - `0 <= i, j, k, l < n`
  - `nums1[i] + nums2[j] + nums3[k] + nums4[l] == 0`
   
  **Example 1:**
  **Input:** nums1 = [1,2], nums2 = [-2,-1], nums3 = [-1,2], nums4 = [0,2]
  **Output:** 2
  **Explanation:**
  The two tuples are:
  1. (0, 0, 0, 1) -> nums1[0] + nums2[0] + nums3[0] + nums4[1] = 1 + (-2) + (-1) + 2 = 0
  2. (1, 1, 0, 0) -> nums1[1] + nums2[1] + nums3[0] + nums4[0] = 2 + (-1) + (-1) + 0 = 0
   
  **Example 2:**
  **Input:** nums1 = [0], nums2 = [0], nums3 = [0], nums4 = [0]
  **Output:** 1
   
  **Constraints:**
   
  - `n == nums1.length`
  - `n == nums2.length`
  - `n == nums3.length`
  - `n == nums4.length`
  - `1 <= n <= 200`
  - `-228<= nums1[i], nums2[i], nums3[i], nums4[i] <= 228`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/4sum-ii/description/)

- Topics: Arrays, Hash Table

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Counter approach
Using Counter to count the a+b pairs from nums1 & nums2 and then summing the counts for every matching -(c+d) from nums3 & nums4

- Time Complexity: O(n^2). We have 2 nested loops to count sums, and another 2 nested loops to find complements.
- Space Complexity: O(n^2) for the hashmap. There could be up to O(n^2) distinct a + b keys.

In [ ]:
from collections import Counter
from typing import List


class Solution:
    def fourSumCount(self, nums1: List[int], nums2: List[int], nums3: List[int], nums4: List[int]) -> int:
        AB = Counter(a+b for a in nums1 for b in nums2)
        return sum(AB[-(c+d)] for c in nums3 for d in nums4)

### Solution 1.1, DefaultDict approach

- Time Complexity: O(n^2). We have 2 nested loops to count sums, and another 2 nested loops to find complements.
- Space Complexity: O(n^2) for the hashmap. There could be up to O(n^2) distinct a + b keys.

In [ ]:
from collections import defaultdict


class Solution:
    def fourSumCount(self, nums1: List[int], nums2: List[int], nums3: List[int], nums4: List[int]) -> int:
        count = defaultdict(int)
        # Count all sums from first two arrays
        for a in nums1:
            for b in nums2:
                count[a + b] += 1
        # Check complementary sums from last two arrays
        return sum(count[-(c + d)] for c in nums3 for d in nums4)

In [ ]:
myobj = Solution()
A = [1, 2]
B = [-2, -1]
C = [-1, 2]
D = [0, 2]
print(myobj.fourSumCount(A, B, C, D))

### Solution 2, General kSum II solution

After you solve 4Sum II, an interviewer can follow-up with 5Sum II, 6Sum II, and so on. What they are really expecting is a generalized solution for k input arrays. Fortunately, the hashmap approach can be easily extended to handle more than 4 arrays.

Above, we divided four arrays into two equal groups and processed each group independently. Same way, we will divide k arrays into two groups. For the first group, we will have ⌊k/2⌋ nested loops to count sums. Another ⌈k/2⌈ nested loops will enumerate arrays in the second group to count sums as before. Finally, we count the pairs from the two groups with sums as 0.

how um_count() method works:
Given [[1, 2], [-2, -1]]:

Start: res = {0: 1}
After array [1, 2]: res = {1: 1, 2: 1} (we can make sum 1 or sum 2)
After array [-2, -1]:
From sum 1: 1 + (-2) = -1, 1 + (-1) = 0
From sum 2: 2 + (-2) = 0, 2 + (-1) = 1
Result: res = {-1: 1, 0: 2, 1: 1} (sum -1 in 1 way, sum 0 in 2 ways, sum 1 in 1 way)

- Time Complexity: O(n^⌈k/2⌉), or O(n^2) for 4Sum II. 
  - We have to enumerate over at most n⌊k/2⌋ sums in the left group and n⌈k/2⌉ sums in the right group. Finally, we just need to check On⌊k/2⌋ sums in the left group and search if their negated number exists in the right group.
- Space Complexity: O(n^⌈k/2⌉)
  - Similarly, we create a HashMap for each group to store all sums, which contains at most n⌈k/2⌉ keys.

In [ ]:
from collections import Counter


class Solution:
    def fourSumCount(self, A: List[int], B: List[int], C: List[int], D: List[int]) -> int:
        
        # calculates all possible sums that can be formed by selecting one element from each array in the input list, along with how many ways each sum can be formed.
        def sum_count(lsts: List[List[int]]) -> Counter:
            # Start with sum 0, which has 1 way (selecting nothing yet)
            total_count = Counter({0: 1})
            for lst in lsts:
                temp = Counter()
                for no in lst:
                    for total in total_count:
                        temp[total + no] += total_count[total]
                total_count = temp
            return total_count

        lsts = [A, B, C, D]
        
        k = len(lsts)
        
        left = sum_count(lsts[:k//2])
        
        right = sum_count(lsts[k//2:])
        
        # computes the total number of valid tuples by summing the products of counts 
        # for complementary sums between the left and right groups
        # it counts all possible combinations, not just individual matches
        # If left[s] = 3 and right[-s] = 2, then 3 * 2 = 6 represents:
        # 3 different ways to form sum s from arrays A and B
        # 2 different ways to form sum -s from arrays C and D
        # 6 different tuples total (each of the 3 left combinations can pair with each of the 2 right combinations)
        return sum(left[s]*right[-s] for s in left)